In [20]:
# Load hotels

In [21]:
# For loop goign through each hotel, checking if it has a match in traffic dataset,
# then creates a hash of the url, then checks if hash and hotel id are already in downloaded
# file, then calls download image function, then saves images the to downloaded file

In [22]:
# NEED FUNCTIONS FOR: 
# 1. checking if it has a match in traffic dataset
# 2. creating a hash of the url
# 3. checking if the hash and hotel id are already downloaded in file
# 4. downloading the images
# 5. saves the image to the download file

In [160]:
import numpy as np
import pandas as pd
import os
import hashlib
from scipy.spatial.distance import cdist
from rapidfuzz import fuzz
from rapidfuzz import utils
import json
import requests
from urllib.parse import urlparse

In [24]:
# Splits the gps column in the booking.com data
def split_gps_column(sharded_data):
    if 'gps' in sharded_data.columns:
        # Split the 'gps' column on the delimiter (assuming comma, adjust if necessary)
        sharded_data[['latitude', 'longitude']] = sharded_data['gps'].str.split(',', expand=True)
        
        # Convert latitude and longitude to numeric types
        sharded_data['latitude'] = pd.to_numeric(sharded_data['latitude'], errors='coerce')
        sharded_data['longitude'] = pd.to_numeric(sharded_data['longitude'], errors='coerce')

        # Drop the 'gps' column after splitting
        sharded_data.drop('gps', axis=1, inplace=True)
    
    return sharded_data

In [25]:
def remove_after_near(s):
    index = s.lower().find("near")
    return s[:index].strip() if index != -1 else s

In [26]:
# Load the TraffickCam data
def loadTraffickCam(path):
    with open(path, 'r', encoding='utf-8') as file:  # Specify utf-8 encoding
        txt_data = file.readlines()
    
    txt_data = [line.strip().split('\t') for line in txt_data]

    columns = ['number', 'hotel_name', 'latitude', 'longitude', 'places_id']
    txt_data_df = pd.DataFrame(txt_data, columns=columns)

    txt_data_df['longitude'] = pd.to_numeric(txt_data_df['longitude'], errors='coerce')
    txt_data_df['latitude'] = pd.to_numeric(txt_data_df['latitude'], errors='coerce')
    txt_data_df.dropna(subset=['longitude', 'latitude'], inplace=True)

    return txt_data_df

In [27]:
# Load the sharded booking.com data
def load_sharded_dataset(root_dir):
    all_data = []
    print("after making list")
    try:
        print("after try")
        for root, dirs, files in os.walk(root_dir):
            #print("in for loop")
            #print(f"Scanning directory: {root}")  # Log current directory being scanned
            
            if not files:
                print(f"No files found in directory: {root}")
                continue  # Skip directories with no files
            
            for file in files:
                file_path = os.path.join(root, file)
                #print(f"Attempting to load file: {file_path}")  # Log which file is being loaded
                
                # Check file extension and load accordingly
                if file.endswith('.csv'):
                    try:
                        data = pd.read_csv(file_path)
                        all_data.append(data)
                        #print(f"Successfully loaded {file_path} with {len(data)} records.")
                    except Exception as e:
                        print(f"Error loading {file_path}: {e}")
                elif file.endswith('.json'):
                    try:
                        data = pd.read_json(file_path)
                        all_data.append(data)
                        #print(f"Successfully loaded {file_path} with {len(data)} records.")
                    except Exception as e:
                        print(f"Error loading {file_path}: {e}")
                else:
                    print(f"Skipping unsupported file format: {file_path}")
    
    except Exception as e:
        print(f"Exception occurred during directory traversal: {e}")
    
    # Ensure we're returning a single combined DataFrame
    if all_data:
        combined_data = pd.concat(all_data, ignore_index=True)
        combined_data.drop_duplicates(inplace=True, keep = 'first')
        combined_data.drop('images', axis=1, inplace=True)
        combined_data['hotel_name'] = combined_data['hotel_name'].str.replace('-', ' ')
        combined_data.drop_duplicates(keep = 'first', inplace=True)
        print(f"Loaded {len(all_data)} files, combined into one DataFrame.")
        split_gps_column(combined_data)
        return combined_data
    else:
        print("No data loaded.")
        return None
    
    

In [28]:
import hashlib

# Creates the has of the booking.com url
def hash_url(url):
    b_url = str.encode(str(url)) # turns the url into bytes in order to be hashed
    hash = hashlib.md5(b_url).hexdigest() # Hashes the byte-form url using md5 algorithm
    
    return hash

In [93]:
# Clean hotel names
def clean_hotel_names(booking_df, traffic_df):
    bookinglatlng = np.vstack([booking_df["latitude"].to_numpy(), booking_df["longitude"].to_numpy()]).T
    bookingHotelNames = booking_df["hotel_name"].fillna("").to_numpy().astype(str)
    bookingHotelURL = booking_df["url"].fillna("").to_numpy().astype(str)
    
    trafficlatlng = np.vstack([traffic_df["latitude"].to_numpy(), traffic_df["longitude"].to_numpy()]).T
    trafficHotelNames = traffic_df["hotel_name"].fillna("").to_numpy().astype(str)
    trafficHotelID = traffic_df["places_id"].fillna("").to_numpy().astype(str)

    # List of phrases to omit
    phrases_to_omit = ["hotel", "motel", "bed and breakfast", "inn ", "&", "suites", " and", " by", " or", 
                    " the", " an", "ihg", " at", " area", " north", " south", " east", "west "]

    # Convert to lowercase
    cleanedBookingHotelNames = np.char.lower(bookingHotelNames)
    cleanedTrafficHotelNames = np.char.lower(trafficHotelNames)

    # Remove punctuation
    unwanted_chars = "()-.,!?"
    translation_table = str.maketrans('', '', unwanted_chars)
    cleanedBookingHotelNames = np.char.translate(cleanedBookingHotelNames, translation_table)
    cleanedTrafficHotelNames = np.char.translate(cleanedTrafficHotelNames, translation_table)

    # Remove phrases
    for phrase in phrases_to_omit:
        cleanedBookingHotelNames = np.char.replace(cleanedBookingHotelNames, phrase, "")
        cleanedTrafficHotelNames = np.char.replace(cleanedTrafficHotelNames, phrase, "")

    # Remove double spaces
    cleanedBookingHotelNames = np.char.replace(cleanedBookingHotelNames, '  ', ' ')
    cleanedTrafficHotelNames = np.char.replace(cleanedTrafficHotelNames, '  ', ' ')

    # Remove words after "near"
    remove_after_near_vec = np.vectorize(remove_after_near)
    cleanedBookingHotelNames = remove_after_near_vec(cleanedBookingHotelNames)
    cleanedTrafficHotelNames = remove_after_near_vec(cleanedTrafficHotelNames)

    # Strip whitespace
    cleanedBookingHotelNames = np.char.strip(cleanedBookingHotelNames)
    cleanedTrafficHotelNames = np.char.strip(cleanedTrafficHotelNames)
    print(type(cleanedBookingHotelNames))
    # Create final DataFrames
    booking_df_final = pd.DataFrame({
        "cleanedHotelName": cleanedBookingHotelNames,
        "latitude": bookinglatlng[:, 0],
        "longitude": bookinglatlng[:, 1],
        "url": bookingHotelURL
    })

    traffic_df_final = pd.DataFrame({
        "cleanedHotelName": cleanedTrafficHotelNames,
        "latitude": trafficlatlng[:, 0],
        "longitude": trafficlatlng[:, 1],
        "places_id": trafficHotelID
    })

    return booking_df_final, traffic_df_final


In [31]:
def find_distances(booking_df, traffic_df):
    # Initialize arrays to store the 10 closest indices and distances for each point

    latlon_small = np.vstack([np.array(booking_df["latitude"]), np.array(booking_df["longitude"])]).T # Now (3906, 2)
    latlon_large = np.vstack([np.array(traffic_df["latitude"]), np.array(traffic_df["longitude"])]).T  # Now (625138, 2)
    # Compute the pairwise Euclidean distances

    #hotel_name_small = bookingHotelNames.T
    hotel_name_small = np.vstack([np.array(booking_df["cleanedHotelName"])]).T
    hotel_name_large = np.vstack([np.array(traffic_df["cleanedHotelName"])]).T

    booking_urls = np.vstack([np.array(booking_df["url"])]).T
    traffic_places_id = np.vstack([np.array(traffic_df["places_id"])]).T

    unique_latlon_large = np.unique(latlon_large, axis=0)

    num_small = latlon_small.shape[0]

    closest_indices = []
    closest_distances = []
    similarity_scores = []

    distance_threshold = 0.0045

    # Loop through each point in the smaller array
    for i in range(num_small):
        # Compute distances between the ith point in latlon_small and all points in latlon_large
        distances = cdist(latlon_small[i:i+1], latlon_large).flatten()  # distances shape (625138,)

        # Filter indices where the distance is within 500 meters (threshold)
        within_threshold_idx = np.where(distances <= distance_threshold)[0]
        
        # Store the closest indices and corresponding distances
        closest_indices.append(within_threshold_idx)
        closest_distances.append(distances[within_threshold_idx])

        print(f"Points within 500 meters of point {i}: {within_threshold_idx}, Distances: {distances[within_threshold_idx]}")

        ref_hotel_name = hotel_name_small[i].flatten()

        # Compute similarity scores for points within the threshold
        similarity_scores_for_point = []
        for j in range(len(within_threshold_idx)):
            closest_hotel_name = hotel_name_large[within_threshold_idx[j]].flatten()

            # Compute the similarity score using fuzzy string matching
            similarity_score = fuzz.token_set_ratio(str(ref_hotel_name), str(closest_hotel_name), processor=utils.default_process)
            similarity_scores_for_point.append(similarity_score)

            if similarity_score >= 85.1:
                print(f"Similarity score between '{ref_hotel_name}' and '{closest_hotel_name}': {similarity_score}")
        
        similarity_scores.append(similarity_scores_for_point)

        print("-----------")

        if i > 250:
            break

In [108]:
def match_hotels(booking_df, traffic_df):

    # Initialize arrays to store the 10 closest indices and distances for each point

    latlon_small = np.vstack([np.array(booking_df["latitude"]), np.array(booking_df["longitude"])]).T # Now (3906, 2)
    latlon_large = np.vstack([np.array(traffic_df["latitude"]), np.array(traffic_df["longitude"])]).T  # Now (625138, 2)
    # Compute the pairwise Euclidean distances

    #hotel_name_small = bookingHotelNames.T
    hotel_name_small = np.vstack([np.array(booking_df["cleanedHotelName"])]).T
    hotel_name_large = np.vstack([np.array(traffic_df["cleanedHotelName"])]).T

    booking_urls = np.vstack([np.array(booking_df["url"])]).T
    traffic_places_id = np.vstack([np.array(traffic_df["places_id"])]).T

    unique_latlon_large = np.unique(latlon_large, axis=0)

    num_small = latlon_small.shape[0]

    closest_indices = []
    closest_distances = []
    similarity_scores = []

    distance_threshold = 0.0045

    # Loop through each point in the smaller array
    for i in range(num_small):
        # Compute distances between the ith point in latlon_small and all points in latlon_large
        distances = cdist(latlon_small[i:i+1], latlon_large).flatten()  # distances shape (625138,)

        # Filter indices where the distance is within 500 meters (threshold)
        within_threshold_idx = np.where(distances <= distance_threshold)[0]
        
        # Store the closest indices and corresponding distances
        closest_indices.append(within_threshold_idx)
        closest_distances.append(distances[within_threshold_idx])

        #print(f"Points within 500 meters of point {i}: {within_threshold_idx}, Distances: {distances[within_threshold_idx]}")

        ref_hotel_name = hotel_name_small[i].flatten()

        # Compute similarity scores for points within the threshold
        similarity_scores_for_point = []
        for j in range(len(within_threshold_idx)):
            closest_hotel_name = hotel_name_large[within_threshold_idx[j]].flatten()
            
            # Compute the similarity score using fuzzy string matching
            similarity_score = fuzz.token_set_ratio(str(ref_hotel_name), str(closest_hotel_name), processor=utils.default_process)
            similarity_scores_for_point.append(similarity_score)

            '''if similarity_score >= 85.1:
                print(f"Similarity score between '{ref_hotel_name}' and '{closest_hotel_name}': {similarity_score}")'''
        
        similarity_scores.append(similarity_scores_for_point)

        #print("-----------")

    # Ensure closest_indices and closest_distances have the same length as num_small
    #num_small = 252
    '''if len(closest_indices) != num_small or len(closest_distances) != num_small:
        print("Error: Length mismatch between closest_indices/closest_distances and num_small.")
        print(f"Expected length: {num_small}, closest_indices length: {len(closest_indices)}, closest_distances length: {len(closest_distances)}")
        raise ValueError("Mismatch in lengths between closest_indices/closest_distances and num_small.")
    '''

    # Initialize the dictionary to store matches
    matches = {}

    # Define the similarity score threshold for a match
    similarity_threshold = 100
    print("here")

    # Loop through each point in the smaller dataset (bookingData)
    for i in range(num_small):
        #print("here2")
        # Safety check in case of indexing issues
        if i >= len(closest_indices) or i >= len(closest_distances):
            print(f"Skipping index {i} due to out-of-range access.")
            continue

        booking_hotel_name = str(hotel_name_small[i][0])  # Get the booking hotel name as a string

        # Check if there are matches within the threshold for this booking hotel
        if len(closest_indices[i]) == 0:
            continue

        # List to hold matches for the current booking hotel
        matched_hotels = []

        # Iterate over each matching traffic hotel within the distance threshold
        for j, idx in enumerate(closest_indices[i]):
            #print("here3")
            similarity_score = similarity_scores[i][j]
          
            # Only add matches that meet or exceed the similarity threshold
            if similarity_score >= similarity_threshold:
                # Prepare match details
                traffic_hotel_name = str(hotel_name_large[idx][0])  # Traffic hotel name as string
                lat, lon = latlon_large[idx]  # Traffic hotel coordinates
                dist = closest_distances[i][j]  # Distance to booking hotel
                b_url = booking_urls[i]

                # Append the match details to the list
                matched_hotels.append({
                    "traffic_hotel_name": traffic_hotel_name,
                    "url": b_url,
                    "latitude": lat,
                    "longitude": lon,
                    "similarity_score": similarity_score,
                    "distance": dist
                })

        # Only add the booking hotel to the dictionary if there are matches
        if matched_hotels:
            matches[booking_hotel_name] = matched_hotels
            
    matched_booking_hotels = []
    # Print out the first few matches for inspection
    for booking_hotel, traffic_matches in list(matches.items())[:100]:
        print(f"Booking Hotel: {booking_hotel}")
        for match in traffic_matches:
            print(f"  Matched Traffic Hotel: {match['traffic_hotel_name']}, "
                f"Latitude: {match['latitude']}, Longitude: {match['longitude']}, "
                f"Score: {match['similarity_score']}, Distance: {match['distance']}")
        matched_booking_hotels.append(booking_hotel)
        print("-----")

    return matches


In [143]:
# Checks to see if the url has has already had its images downloaded from it
def predownload_check(hash_to_check):
    # Ensure the file exists
    file_path = "downloaded_hashes.txt"
    if not os.path.exists(file_path):
        with open(file_path, 'w') as f:
            pass  # Create an empty file
    
    # Read the file and check for the hash
    with open(file_path, 'r') as f:
        lines = f.readlines()
    
    check = hash_to_check in (line.strip() for line in lines)
    print(check)
    return check

In [162]:
def download_image(url, save_directory):
    try:
        # Get the file name from the URL
        parsed_url = urlparse(url)
        print("parsed url")
        file_name = os.path.basename(parsed_url.path)
        print("made file name")
        save_path = os.path.join(save_directory, file_name)
        print("saved path")

        # Ensure the save directory exists
        os.makedirs(save_directory, exist_ok=True)
        print("made directory")

        # Download the image
        response = requests.get(url, stream=True)
        print("requested page")
        response.raise_for_status()  # Raise an error for HTTP errors
        print("raised for status")

        # Save the image
        with open(save_path, 'wb') as image_file:
            for chunk in response.iter_content(1024):
                image_file.write(chunk)
        print(f"Downloaded: {url} -> {save_path}")
    except Exception as e:
        print(f"Failed to download {url}: {e}")

START OF MAIN

In [33]:
#Load TraffickCam Data
traffickCamData = loadTraffickCam("hotel_info_with_places_id.txt")
traffickCamData.head

<bound method NDFrame.head of         number                                         hotel_name   latitude  \
0          390                                  Super 8 Frederick  39.402160   
1          391  Extended Stay America - Fairbanks - Old Airpor...  64.835380   
2          392                Hilton Hangzhou Qiandao Lake Resort  29.608190   
3          393                                      Taj Lands End  19.043910   
4          394        Helzear Montparnasse Rive Gauche Apartments  48.834660   
...        ...                                                ...        ...   
625133  694001   Hyatt Regency Lake Washington at Seattle’s So...  47.503043   
625134  694002                                 The Beverly Hilton  34.066370   
625135  694003               Homewood Suites by Hilton Ronkonkoma  40.812062   
625136  694004                                  Choctaw Durant OK  33.958543   
625137  694005                                            ???????  24.125850   

         

In [34]:
# Load booking.com Data
root_dir = 'C:/Users/lucas/Desktop/VisLab/country/United-States-of-America'
bookingData = load_sharded_dataset(root_dir)
bookingData.info

after making list
after try
No files found in directory: C:/Users/lucas/Desktop/VisLab/country/United-States-of-America
No files found in directory: C:/Users/lucas/Desktop/VisLab/country/United-States-of-America\Alabama
No files found in directory: C:/Users/lucas/Desktop/VisLab/country/United-States-of-America\Alaska
No files found in directory: C:/Users/lucas/Desktop/VisLab/country/United-States-of-America\Arizona
No files found in directory: C:/Users/lucas/Desktop/VisLab/country/United-States-of-America\Arkansas
No files found in directory: C:/Users/lucas/Desktop/VisLab/country/United-States-of-America\California
No files found in directory: C:/Users/lucas/Desktop/VisLab/country/United-States-of-America\Colorado
No files found in directory: C:/Users/lucas/Desktop/VisLab/country/United-States-of-America\Connecticut
No files found in directory: C:/Users/lucas/Desktop/VisLab/country/United-States-of-America\Delaware
No files found in directory: C:/Users/lucas/Desktop/VisLab/country/Unit

<bound method DataFrame.info of                                                hotel_name date_accessed  \
0       Candlewood Suites Alabaster, an IHG Hotel (Hotel)    2024-08-26   
67      Fairfield by Marriott Inn & Suites Albertville...    2024-10-15   
102     Microtel Inn & Suites by Wyndham Albertville (...    2024-10-27   
135     InTown Suites Extended Stay Birmingham AL   Ho...    2024-08-29   
169                              Econo Lodge Boaz (Hotel)    2024-10-27   
...                                                   ...           ...   
255764                 The Hatchet Resort (Resort), Moran    2024-07-01   
255834             Togwotee Mountain Lodge (Lodge), Moran    2024-07-01   
255878    Cobblestone Hotel & Suites   Torrington (Hotel)    2024-09-07   
255971  Holiday Inn Express Hotel & Suites Torrington,...    2024-09-07   
256021                     King's Inn (Hotel), Torrington    2024-09-07   

                                                  address rating  \

In [94]:
cleanBookingData, cleanTrafficData = clean_hotel_names(bookingData, traffickCamData)

<class 'numpy.ndarray'>


In [95]:
print(cleanBookingData['cleanedHotelName'])

0                             candlewood alabaster
1                  fairfield marriott  albertville
2                    microtel  wyndham albertville
3       intown extended stay birmingham al  hoover
4                                 econo lodge boaz
                           ...                    
4938               the hatchet resort resort moran
4939           togwotee mountain lodge lodge moran
4940                      cobblestone   torrington
4941                   holiday express  torrington
4942                             king's torrington
Name: cleanedHotelName, Length: 4943, dtype: object


In [96]:
cleanTrafficData.head

<bound method NDFrame.head of                                        cleanedHotelName   latitude  \
0                                     super 8 frederick  39.402160   
1       extended stay america fairbanks old airport way  64.835380   
2                   hilton hangzhou qiandao lake resort  29.608190   
3                                         taj lands end  19.043910   
4           helzear montparnasse rive gauche apartments  48.834660   
...                                                 ...        ...   
625133      hyatt regency lake washington seattle’sport  47.503043   
625134                               the beverly hilton  34.066370   
625135                       homewood hilton ronkonkoma  40.812062   
625136                                choctaw durant ok  33.958543   
625137                                                   24.125850   

         longitude                    places_id  
0       -77.408420  ChIJCWJjR0HayYkRAfWSLQl85IQ  
1      -147.823300  ChIJqbFJV

In [109]:
matches = match_hotels(cleanBookingData, cleanTrafficData)

here
Booking Hotel: candlewood alabaster
  Matched Traffic Hotel: candlewood alabaster, Latitude: 33.2329, Longitude: -86.80002, Score: 100.0, Distance: 0.0007507592756763593
  Matched Traffic Hotel: candlewood alabaster alabama, Latitude: 33.2329592, Longitude: -86.7993574, Score: 100.0, Distance: 0.00013271018801404001
-----
Booking Hotel: microtel  wyndham albertville
  Matched Traffic Hotel: microtel  wyndham albertville, Latitude: 34.27784, Longitude: -86.20232, Score: 100.0, Distance: 0.001346336758240169
-----
Booking Hotel: econo lodge boaz
  Matched Traffic Hotel: econo lodge, Latitude: 34.19772, Longitude: -86.15006, Score: 100.0, Distance: 0.001093845053010089
-----
Booking Hotel: key  boaz
  Matched Traffic Hotel: key boaz al, Latitude: 34.19852, Longitude: -86.15788, Score: 100.0, Distance: 0.0003899357941533613
-----
Booking Hotel: hampton calera
  Matched Traffic Hotel: hampton calera, Latitude: 33.14882, Longitude: -86.75135, Score: 100.0, Distance: 0.001975749005441097

In [110]:
print(matches)

{'candlewood alabaster': [{'traffic_hotel_name': 'candlewood alabaster', 'url': array(['https://www.booking.com/hotel/us/candlewood-suites-alabaster.html?aid=304142&label=gen173nr-1FCAMo7AFCCWFsYWJhc3RlckgzWARonQKIAQGYATG4AQfIAQ_YAQHoAQH4AQKIAgGoAgO4AqOtsrYGwAIB0gIkMjdmOGMwMjMtNzk1Zi00MmRlLTgwYzgtYWE5ZWI1MThiNjE42AIF4AIB&sid=4f648819a52e44b88635f568c13b1d71&dest_id=20000038&dest_type=city&dist=0&group_adults=2&group_children=0&hapos=2&hpos=2&no_rooms=1&req_adults=2&req_children=0&room1=A%2CA&sb_price_type=total&sr_order=popularity&srepoch=1724684074&srpvid=7a6268cf5f820527&type=total&ucfs=1&activeTab=photosGallery#close-lightbox'],
      dtype=object), 'latitude': 33.2329, 'longitude': -86.80002, 'similarity_score': 100.0, 'distance': 0.0007507592756763593}, {'traffic_hotel_name': 'candlewood alabaster alabama', 'url': array(['https://www.booking.com/hotel/us/candlewood-suites-alabaster.html?aid=304142&label=gen173nr-1FCAMo7AFCCWFsYWJhc3RlckgzWARonQKIAQGYATG4AQfIAQ_YAQHoAQH4AQKIAgGoAgO

In [150]:
hash_value = hashlib.md5("https://www.booking.com/hotel/us/candlewood-suites-alabaster.html?aid=304142&label=gen173nr-1FCAMo7AFCCWFsYWJhc3RlckgzWARonQKIAQGYATG4AQfIAQ_YAQHoAQH4AQKIAgGoAgO4AqOtsrYGwAIB0gIkMjdmOGMwMjMtNzk1Zi00MmRlLTgwYzgtYWE5ZWI1MThiNjE42AIF4AIB&sid=4f648819a52e44b88635f568c13b1d71&dest_id=20000038&dest_type=city&dist=0&group_adults=2&group_children=0&hapos=2&hpos=2&no_rooms=1&req_adults=2&req_children=0&room1=A%2CA&sb_price_type=total&sr_order=popularity&srepoch=1724684074&srpvid=7a6268cf5f820527&type=total&ucfs=1&activeTab=photosGallery#close-lightbox".encode('utf-8')).hexdigest()

print(hash_value)

hash_check_bool = predownload_check(hash_value)
print(hash_check_bool)

b2ee652b30766d8a59c7f62d8c0e14c5
False
False


In [ ]:
for key, sub_dict in matches.items():
        for item in sub_dict:
            url = item.get("url", "No URL Found")
            #print(f"{key} - URL: {url}")
            url = str(url)
            hash = hashlib.md5(url.encode('utf-8')).hexdigest()
            check = predownload_check(hash)
            if check == True:
                break
            else:
                # Directory to save downloaded images
                image_directory = 'C:/Users/lucas/Desktop/TraffickCam_Nearby_Search/booking_images'
                # Ensure the image directory exists
                os.makedirs(image_directory, exist_ok=True)
                
                download_image(url, image_directory)




True
False
parsed url
made file name
saved path
made directory
Failed to download ['https://www.booking.com/hotel/us/microtel-inn-albertville.html']: No connection adapters were found for "['https://www.booking.com/hotel/us/microtel-inn-albertville.html']"
False
parsed url
made file name
saved path
made directory
Failed to download ['https://www.booking.com/hotel/us/econo-lodge-boaz.html']: No connection adapters were found for "['https://www.booking.com/hotel/us/econo-lodge-boaz.html']"
False
parsed url
made file name
saved path
made directory
Failed to download ['https://www.booking.com/hotel/us/key-west-inn-boaz.html']: No connection adapters were found for "['https://www.booking.com/hotel/us/key-west-inn-boaz.html']"
False
parsed url
made file name
saved path
made directory
Failed to download ['https://www.booking.com/hotel/us/hampton-inn-calera.html?aid=304142&label=gen173nr-1FCAMo7AFCCWFsYWJhc3RlckgzWARonQKIAQGYATG4AQfIAQ_YAQHoAQH4AQKIAgGoAgO4AqOtsrYGwAIB0gIkMjdmOGMwMjMtNzk1Zi00M

KeyboardInterrupt: 